In [9]:
from collections import defaultdict
import expt
import pandas as pd

In [80]:
runs_dir = "/dfs/scratch1/ranjanr/runs/cs336/2024-05-03"
df = expt.scan(runs_dir)
df

  0%|          | 0/21 [00:00<?, ?it/s]

!rm -r /dfs/scratch1/ranjanr/runs/cs336/2024-05-03/1714865005480430048  # dev


,script_path,runs_dir,dev,lm_size,vocab_size,context_length,d_model,num_layers,num_heads,d_ff,attn_pdrop,residual_pdrop,batch_size,warmup_steps,benchmark_steps,amp,done,compile
1714795781928288391,/lfs/ampere8/0/ranjanr/cs336-a2/cs336-systems/...,/dfs/scratch1/ranjanr/runs/cs336/2024-05-03,0,small,10000,128,768,12,12,3072,0.0,0.0,16,1,5,0,True,NaN
1714795790379185206,/lfs/ampere8/0/ranjanr/cs336-a2/cs336-systems/...,/dfs/scratch1/ranjanr/runs/cs336/2024-05-03,0,small,10000,128,768,12,12,3072,0.0,0.0,16,1,5,1,True,NaN
1714795796741125722,/lfs/ampere8/0/ranjanr/cs336-a2/cs336-systems/...,/dfs/scratch1/ranjanr/runs/cs336/2024-05-03,0,medium,10000,128,1024,24,16,4096,0.0,0.0,16,1,5,0,True,NaN
1714795860377535384,/lfs/ampere8/0/ranjanr/cs336-a2/cs336-systems/...,/dfs/scratch1/ranjanr/runs/cs336/2024-05-03,0,medium,10000,128,1024,24,16,4096,0.0,0.0,16,1,5,1,True,NaN
1714795869118342729,/lfs/ampere8/0/ranjanr/cs336-a2/cs336-systems/...,/dfs/scratch1/ranjanr/runs/cs336/2024-05-03,0,large,10000,128,1280,36,20,5120,0.0,0.0,16,1,5,0,True,NaN
1714795889761582337,/lfs/ampere8/0/ranjanr/cs336-a2/cs336-systems/...,/dfs/scratch1/ranjanr/runs/cs336/2024-05-03,0,large,10000,128,1280,36,20,5120,0.0,0.0,16,1,5,1,True,NaN
1714795911492260796,/lfs/ampere8/0/ranjanr/cs336-a2/cs336-systems/...,/dfs/scratch1/ranjanr/runs/cs336/2024-05-03,0,xl,10000,128,1600,48,25,6400,0.0,0.0,16,1,5,0,True,NaN
1714795949038588417,/lfs/ampere8/0/ranjanr/cs336-a2/cs336-systems/...,/dfs/scratch1/ranjanr/runs/cs336/2024-05-03,0,xl,10000,128,1600,48,25,6400,0.0,0.0,16,1,5,1,True,NaN
1714795971807577187,/lfs/ampere8/0/ranjanr/cs336-a2/cs336-systems/...,/dfs/scratch1/ranjanr/runs/cs336/2024-05-03,0,2.7b,10000,128,2560,32,32,10240,0.0,0.0,16,1,5,0,True,NaN
1714796024046762300,/lfs/ampere8/0/ranjanr/cs336-a2/cs336-systems/...,/dfs/scratch1/ranjanr/runs/cs336/2024-05-03,0,2.7b,10000,128,2560,32,32,10240,0.0,0.0,16,1,5,1,True,NaN


In [79]:
"compile" in df.columns

False

In [35]:
def make_pd_tab(inference_only):
    tab = defaultdict(dict)
    for lm_size in [
        "small",
        "medium",
        "large",
        "xl",
        "2.7b",
    ]:
        ts = df.query("lm_size == @lm_size and inference_only == @inference_only and amp.isnull()").index[-1]
        for component in ["param", "activation", "peak"]:
            val = df[f"{component} memory (B)"][ts]
            if isinstance(val, tuple):
                val = val[0]
            tab[lm_size][f"{component} memory (GiB)"] = val / 2**30
    pd_tab = pd.DataFrame(tab)
    return pd_tab

In [38]:
for inference_only in [
    0,
    1,
]:
    print(f"{inference_only=}")
    pd_tab = make_pd_tab(inference_only)
    display(pd_tab.style.format("{:.2f}"))

inference_only=0


,small,medium,large,xl,2.7b
param memory (GiB),0.35,1.16,2.76,5.69,9.60
activation memory (GiB),2.53,6.16,11.26,18.78,19.21
peak memory (GiB),3.58,9.66,19.42,35.66,48.00


inference_only=1


,small,medium,large,xl,2.7b
param memory (GiB),0.35,1.16,2.76,5.69,9.60
activation memory (GiB),0.13,0.17,0.21,0.27,0.42
peak memory (GiB),0.55,1.41,3.05,6.03,10.09


In [39]:
def make_pd_tab(inference_only):
    tab = defaultdict(dict)
    for amp in [
        0,
        1,
    ]:
        lm_size = "2.7b"
        ts = df.query("lm_size == @lm_size and inference_only == @inference_only and amp == @amp").index[-1]
        for component in ["param", "activation", "peak"]:
            val = df[f"{component} memory (B)"][ts]
            if isinstance(val, tuple):
                val = val[0]
            tab[f"{amp=}"][f"{component} memory (GiB)"] = val / 2**30
    pd_tab = pd.DataFrame(tab)
    return pd_tab

In [40]:
for inference_only in [
    0,
    1,
]:
    print(f"{inference_only=}")
    pd_tab = make_pd_tab(inference_only)
    display(pd_tab.style.format("{:.2f}"))

inference_only=0


,amp=0,amp=1
param memory (GiB),9.60,9.60
activation memory (GiB),19.21,17.18
peak memory (GiB),48.00,45.74


inference_only=1


,amp=0,amp=1
param memory (GiB),9.60,9.60
activation memory (GiB),0.42,0.42
peak memory (GiB),10.09,10.09


In [69]:
def make_pd_tab():
    script_path = "/lfs/ampere1/0/ranjanr/cs336-a2/cs336-systems/expt/norm.py"
    tab = defaultdict(dict)
    for d_model in [
        1024,
        2048,
        4096,
        8192,
    ]:
        for norm in [
            "layer",
            "rms",
            "triton_rms",
            "compiled_rms",
        ]:
            ts = df.query("d_model == @d_model and norm == @norm and script_path == @script_path and (backward == 0 or backward.isnull())").index[-1]
            store = expt.PathDict(f"{runs_dir}/{ts}")
            tab[d_model][f"{norm}_norm time (ms)"] = store["time (s)"].mean().item() * 1000
    pd_tab = pd.DataFrame(tab)
    return pd_tab

In [70]:
pd_tab = make_pd_tab()
pd_tab.style.format("{:.3f}").highlight_min(axis=0)

,1024,2048,4096,8192
layer_norm time (ms),0.294,0.543,1.376,2.915
rms_norm time (ms),0.921,1.791,3.524,6.958
triton_rms_norm time (ms),0.335,0.570,1.034,1.959
compiled_rms_norm time (ms),0.303,0.633,1.060,2.587


In [56]:
def make_pd_tab():
    script_path = "/lfs/ampere1/0/ranjanr/cs336-a2/cs336-systems/expt/norm_lm.py"
    tab = defaultdict(dict)
    for lm_size in [
        "small",
        "medium",
        "large",
        "xl",
        "2.7b",
    ]:
        for norm in [
            "layer",
            "rms",
            "triton_rms",
        ]:
            ts = df.query("lm_size == @lm_size and norm == @norm and script_path == @script_path").index[-1]
            store = expt.PathDict(f"{runs_dir}/{ts}")
            tab[lm_size][f"{norm}_norm time (s)"] = store["time (s)"].mean().item()
    pd_tab = pd.DataFrame(tab)
    return pd_tab

In [58]:
pd_tab = make_pd_tab()
pd_tab.style.format("{:.4f}").highlight_min(axis=0)

,small,medium,large,xl,2.7b
layer_norm time (s),0.0325,0.0895,0.1928,0.3998,0.6115
rms_norm time (s),0.0324,0.0908,0.1947,0.4028,0.6143
triton_rms_norm time (s),0.0314,0.0893,0.1920,0.3983,0.6106


In [71]:
def make_pd_tab():
    script_path = "/lfs/ampere1/0/ranjanr/cs336-a2/cs336-systems/expt/norm.py"
    tab = defaultdict(dict)
    for d_model in [
        1024,
        2048,
        4096,
        8192,
    ]:
        for norm in [
            "layer",
            "rms",
            "triton_rms",
            "compiled_rms",
        ]:
            ts = df.query("d_model == @d_model and norm == @norm and script_path == @script_path and backward == 1").index[-1]
            store = expt.PathDict(f"{runs_dir}/{ts}")
            tab[d_model][f"{norm}_norm time (ms)"] = store["time (s)"].mean().item() * 1000
    pd_tab = pd.DataFrame(tab)
    return pd_tab

In [72]:
pd_tab = make_pd_tab()
pd_tab.style.format("{:.3f}").highlight_min(axis=0)

,1024,2048,4096,8192
layer_norm time (ms),1.287,1.920,3.703,8.169
rms_norm time (ms),1.825,3.354,6.359,12.375
triton_rms_norm time (ms),1.504,2.472,4.828,8.942
compiled_rms_norm time (ms),1.583,2.305,3.835,10.867


In [88]:
def make_pd_tab(pass_):
    script_path = "/lfs/ampere1/0/ranjanr/cs336-a2/cs336-systems/expt/compile.py"
    tab = defaultdict(dict)
    for lm_size in [
        "small",
        "medium",
        "large",
        "xl",
        "2.7b",
    ]:
        ctime = {}
        for compile_ in [
            0,
            1
        ]:
            ts = df.query("lm_size == @lm_size and compile == @compile_ and script_path == @script_path").index[-1]
            store = expt.PathDict(f"{runs_dir}/{ts}")
            ctime[compile_] = store[f"{pass_}time"].mean().item()
            tab[lm_size][f"{compile_=} time (s)"] = ctime[compile_]
        tab[lm_size]["speedup"] = ctime[0] / ctime[1]
    pd_tab = pd.DataFrame(tab)
    return pd_tab

In [89]:
for pass_ in [
    "fw_",
    "",
]:
    print(f"{pass_=}")
    pd_tab = make_pd_tab(pass_)
    display(pd_tab.style.format("{:.4f}"))

pass_='fw_'


,small,medium,large,xl,2.7b
compile_=0 time (s),0.0305,0.2154,0.1956,0.4037,0.6135
compile_=1 time (s),0.1627,0.1584,0.1806,0.3704,0.5804
speedup,0.1874,1.3601,1.0830,1.0898,1.0571


pass_=''


,small,medium,large,xl,2.7b
compile_=0 time (s),0.1032,0.4260,0.6563,1.5453,2.0830
compile_=1 time (s),0.2227,0.3439,0.5976,1.2180,1.9587
speedup,0.4634,1.2389,1.0983,1.2687,1.0635
